In [17]:
!code acquire.py

In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from bs4 import BeautifulSoup
import pandas as pd

from env import github_token, github_username

In [2]:
start_link = 'https://github.com/search?q=space&type=Repositories'
filename = 'repos.json'
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}
response = requests.get(start_link, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
main_link = 'https://github.com'
languages = ['JavaScript', 'Python', 'C#', 'Java']
all_blocks = soup.find_all('li', class_='repo-list-item hx_hit-repo d-flex flex-justify-start py-4 public source')
test_links = []
start_link = 'https://github.com/search?q=space&type=Repositories'
#filename = 'repos.json'
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}
response = requests.get(start_link, headers=headers)
all_blocks = soup.find_all('li', class_='repo-list-item hx_hit-repo d-flex flex-justify-start py-4 public source')

for block in all_blocks:
    language = block.find('span', itemprop='programmingLanguage').text
    link = block.find('a', class_='v-align-middle')['href'][1:]
    
    test_links.append(link)


In [4]:
#import acquire

ModuleNotFoundError: No module named 'acquire'

In [5]:
#pd.DataFrame(acquire.scrape_github_data())

NameError: name 'acquire' is not defined

In [3]:
REPOS = test_links

In [4]:
all_blocks = soup.find_all('li', class_='repo-list-item hx_hit-repo d-flex flex-justify-start py-4 public source')

In [97]:
all_blocks[0].find('a', class_='v-align-middle')['href'][1:]

'SpaceVim/SpaceVim'

In [98]:
test_links

['SpaceVim/SpaceVim',
 'syl20bnr/spacemacs',
 'r-spacex/SpaceX-API',
 'KeenSoftwareHouse/SpaceEngineers',
 'a1studmuffin/SpaceshipGenerator',
 'MicrosoftDocs/mslearn-tailspin-spacegame-web',
 'spaceship-prompt/spaceship-prompt',
 'spacedriveapp/spacedrive',
 'SublimeText/Spacegray',
 'sar-gupta/space']

In [22]:
#get_repo_contents('SpaceVim/SpaceVim')

In [7]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }



def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [100]:
REPOS

['SpaceVim/SpaceVim',
 'syl20bnr/spacemacs',
 'r-spacex/SpaceX-API',
 'KeenSoftwareHouse/SpaceEngineers',
 'a1studmuffin/SpaceshipGenerator',
 'MicrosoftDocs/mslearn-tailspin-spacegame-web',
 'spaceship-prompt/spaceship-prompt',
 'spacedriveapp/spacedrive',
 'SublimeText/Spacegray',
 'sar-gupta/space']

In [8]:
d = scrape_github_data()

In [9]:
test_links

['SpaceVim/SpaceVim',
 'syl20bnr/spacemacs',
 'r-spacex/SpaceX-API',
 'KeenSoftwareHouse/SpaceEngineers',
 'a1studmuffin/SpaceshipGenerator',
 'MicrosoftDocs/mslearn-tailspin-spacegame-web',
 'spaceship-prompt/spaceship-prompt',
 'spacedriveapp/spacedrive',
 'SublimeText/Spacegray',
 'sar-gupta/space']

In [10]:
d = pd.DataFrame(d)

In [11]:
d

,repo,language,readme_contents
0,SpaceVim/SpaceVim,Lua,"<h1 align=""center"">\n <a href=""https://spacev..."
1,syl20bnr/spacemacs,Emacs Lisp,"<a name=""top"" id=""fork-destination-box""></a>\n..."
2,r-spacex/SpaceX-API,JavaScript,"<p align=""center""><img src=""https://live.stati..."
3,KeenSoftwareHouse/SpaceEngineers,C#,\nARCHIVED VERSION OF Space Engineers. This ve...
4,a1studmuffin/SpaceshipGenerator,Python,# Spaceship Generator\n\nA Blender script to p...
5,MicrosoftDocs/mslearn-tailspin-spacegame-web,HTML,\r\n# Contributing\r\n\r\nThis project welcome...
6,spaceship-prompt/spaceship-prompt,Shell,[![SWUbanner](https://raw.githubusercontent.co...
7,spacedriveapp/spacedrive,Rust,"<p align=""center"">\n <a href=""#"">\n \n </..."
8,SublimeText/Spacegray,JavaScript,# Spacegray\n\nA set of custom UI themes for S...
9,sar-gupta/space,JavaScript,# S P A C E\n\n![cover](resources/space-readme...


In [13]:
s = BeautifulSoup(d.iloc[1, 2])

In [14]:
for data in s(['style', 'script']):
    data.decompose()
    
' '.join(s.stripped_strings)

'philosophy | for whom? | screenshots | documentation | contribute | achievements | FAQ - - -\n\n**Quick Start**\n\nIf you *don\'t* have an existing Emacs setup and want to run Spacemacs as your\nconfiguration, and if you have all [prerequisites](#prerequisites) installed,\nyou can install Spacemacs with one line:\n\n* shell:\n  ```sh\n  git clone https://github.com/syl20bnr/spacemacs $HOME/.emacs.d\n  ```\n\n* Windows PowerShell:\n\n  ```powershell\n  git clone https://github.com/syl20bnr/spacemacs $HOME/.emacs.d\n  ```\n\n  If `HOME` is not set in environment or in registry:\n\n  ```powershell\n  git clone https://github.com/syl20bnr/spacemacs $env:APPDATA/.emacs.d\n  ```\n\nIf you do have an existing Emacs configuration, look at the [full installation\ninstructions](#install) for other options.\n\n# Introduction\nSpacemacs is a new way of experiencing Emacs -- it\'s a sophisticated and\npolished set-up, focused on *ergonomics*, *mnemonics* and *consistency*.\n\nJust clone and launch

In [15]:
' '.join(s.findAll(text=True))

'\n \n \n \n  logo and links  \n \n \n philosophy \n|\n for whom? \n|\n screenshots \n|\n documentation \n|\n contribute \n|\n achievements \n|\n FAQ \n \n  badges  \n \n \n \n \n \n \n \n \n\n- - -\n\n**Quick Start**\n\nIf you *don\'t* have an existing Emacs setup and want to run Spacemacs as your\nconfiguration, and if you have all [prerequisites](#prerequisites) installed,\nyou can install Spacemacs with one line:\n\n* shell:\n  ```sh\n  git clone https://github.com/syl20bnr/spacemacs $HOME/.emacs.d\n  ```\n\n* Windows PowerShell:\n\n  ```powershell\n  git clone https://github.com/syl20bnr/spacemacs $HOME/.emacs.d\n  ```\n\n  If `HOME` is not set in environment or in registry:\n\n  ```powershell\n  git clone https://github.com/syl20bnr/spacemacs $env:APPDATA/.emacs.d\n  ```\n\nIf you do have an existing Emacs configuration, look at the [full installation\ninstructions](#install) for other options.\n\n# Introduction\nSpacemacs is a new way of experiencing Emacs -- it\'s a sophisticat

[Hello](http://www.google.com)

In [16]:
d.iloc[0, 2]

NameError: name 'd' is not defined

In [67]:
process_repo(test_links[1])

Exception: Error response from github api! status code: 404, response: {"message": "Not Found", "documentation_url": "https://docs.github.com/rest"}

In [29]:
all_blocks[0].find('span', itemprop='programmingLanguage').text

'Lua'

In [30]:
#all_blocks[0]

In [31]:
for i in range(len(all_blocks)):

IndentationError: expected an indented block (2545963203.py, line 1)

In [ ]:
soup.find_all('a', class_='v-align-middle')[0]['href']

In [32]:
soup.find_all('a', class_='v-align-middle')[0]

<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"space","result_position":1,"click_id":77358263,"result":{"id":77358263,"global_relay_id":"MDEwOlJlcG9zaXRvcnk3NzM1ODI2Mw==","model_name":"Repository","url":"https://github.com/SpaceVim/SpaceVim"},"originating_url":"https://github.com/search?q=space&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="3f673f820eb918c884af06688d62e87ae116c5806ee48530a6f3ffcbdbbf76d8" href="/SpaceVim/SpaceVim">SpaceVim/<em>Space</em>Vim</a>

In [33]:
#github_api_request(start_link)

In [34]:
Union?

In [35]:
REPOS = []

In [81]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }



def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)


Exception: Error response from github api! status code: 404, response: {"message": "Not Found", "documentation_url": "https://docs.github.com/rest"}